In [1]:
import os
import pandas as pd

path = 'images/'
label = os.listdir(path)

img_names = os.listdir(path)
docs_df = pd.DataFrame({'img_names':img_names})

docs_df['text'] = ''
docs_df.reset_index(drop=True, inplace=True)
docs_df

In [ ]:
import json

# документы хранятся в json-файле в виде: имя файла фотографии: данные: {список слов текста документа}
with open('original_merged_data.json') as data_file:
    data_dict = json.load(data_file)
data_dict

for key in data_dict:
    text = ' '.join(data_dict[key]['data'])
    docs_df.loc[docs_df['img_names'] == key, ['text']] = text
docs_df = docs_df.loc[docs_df['text'] != '']
docs_df

In [ ]:
with open('ner_labels.json', 'r') as labels_file:
    labels_for_any_docs_dict = json.load(labels_file)
labels_for_any_docs_dict

In [ ]:
entity_types = []
for i in labels_for_any_docs_dict:
    for j in range(len(labels_for_any_docs_dict[i])):
        entity_types.append(labels_for_any_docs_dict[i][j][0])
entity_types = set(entity_types)
print(f' Число сущностей: {len(entity_types)}\n', entity_types)

Одна сущность может называться по-разному, напимер: "KPP" и "кпп", приведу сущности к единым обозначениям

In [ ]:
# для удобства просмотра и редактирования перенесу список сущностей в файл

with open('Сущности.txt', 'w', encoding='utf-8') as f:
    for i in entity_types:
        f.write(i + '\n')

In [ ]:
# после сопоставления в файле взаимозаменяемых сущностей вношу изменения в словарь

translater_dict = {}

with open('Сущности_ред.txt', 'r', encoding='utf-8') as f:
    for i in f.readlines():
        word2translate = i.split()
        translater_dict[word2translate[0]]  = word2translate[1]

for i in labels_for_any_docs_dict:
    for j in range(len(labels_for_any_docs_dict[i])):
        if labels_for_any_docs_dict[i][j][0] in translater_dict:
            labels_for_any_docs_dict[i][j][0] = translater_dict[labels_for_any_docs_dict[i][j][0]]
labels_for_any_docs_dict

### Предобработка данных

##### В данных пристутсвуют выбросы. Поскольку данные были получены на основе разметки человека, предположим, что выбросы можно корректно обработать скриптом, не прибегая к повторной разметке

In [ ]:
from string import punctuation
import re


def get_entry_left(string, sentence):
    for i in range(len(sentence)):
        if string != '' and string != ' ':
            if string in sentence:
                return string
            else:
                string = string[:-1]
        else: return None


def get_entry_right(string, sentence):
    for i in range(len(sentence)):
        if string != '' and string != ' ':
            if string in sentence:
                return string
            else:
                string = string[1:]
        else: return None


def get_max_word(_string, string_):
    if _string != None and string_ != None:
        r = len(_string)
        l = len(string_)
        if max(r, l) == r: return _string
        else: return string_
    elif _string != None and string_ == None:
        return _string
    elif _string == None and string_ != None:
        return string_
    else: return None


def complex_estimation(s, string):
    _s = get_entry_right(s, string)
    s_ = get_entry_left(s, string)
    return get_max_word(_s, s_)


def isfloat(string):
    try:
        float(string)
        return True
    except:
        return False

# фильтр нерелевантных значений для каждой сущности
# оценка некоторых пороговых значений для длины сущности основана на личном опыте работы с разметкой ...
# ... имеющихся в данной работе данных
pattern = '[%s]+' % re.escape(punctuation)
def bad_word(entity_type, entity):
    var = False
    if entity_type == 'ADDRESSBUYER':
        if len(entity) <= 21: var = True
    if entity_type == 'ADDRESSSHIPPER':
        if len(entity) <= 4: var = True
    if entity_type == 'ADDRESSSELLER':
        if len(entity) <= 16: var = True
    if entity_type == 'AMOUNT':
        if isfloat(entity) == False: var = True
        elif re.match(r'[0]\d+', entity) != None: var = True
    if entity_type == 'BPINN':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
        elif len(entity) != 10: var = True
    if entity_type == 'BPKPP':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
        elif len(entity) != 10: var = True
    if entity_type == 'BPNAME':
        if len(entity) < 9: var = True
        elif len(entity) < 10 and entity in ['ОБЩЕСТВО С', 'Общество ', 'ОБЩЕСТВО ', 'Стоимость']: var = True
    if entity_type == 'CONTENT':
        if len(entity) < 3 or entity in ['ливо', ' 116']: var = True
    if entity_type == 'CURRENCY':
        if len(entity) < 8: var = True
    if entity_type == 'CURRENCYKOD':
        if entity.isalpha(): var = True
    if entity_type == 'DOCDATE':
        if len(entity) < 10: var = True
        if ''.join(entity.split()).isalpha(): var = True
    if entity_type == 'DOGDATE':
        if ''.join(entity.split()).isalpha(): var = True
        elif len(entity) < 4: var = True
    if entity_type == 'DOGNUM':
        if ''.join(entity.split()).isalpha(): var = True
    if entity_type == 'INNBUYER':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
        elif len(entity) != 10: var = True
    if entity_type == 'KPPBUYER':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
        elif len(entity) != 10: var = True
    if entity_type == 'MEASUREUNIT':
        if len(re.sub(pattern, '', ''.join(entity.split()))) == 1: var = True
    if entity_type == 'NAMEBUYER':
        if len(entity) < 11: var = True
    if entity_type == 'PRICE':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
    if entity_type == 'SFNUM':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
        elif ''.join(entity.split()) in punctuation: var = True
    if entity_type == 'SHIPPERNAME':
        if len(entity) < 5: var = True
        elif len(entity) < 11 and entity in ['Акционерно', 'Общество ', 'ОБЩЕСТВО ', 'Стоимость']: var = True
    if entity_type == 'UNITPRICE':
        if isfloat(entity) == False: var = True
        elif re.match(r'[.]\d+', entity) != None: var = True
        elif re.match(r'[0]\d+', entity) != None: var = True

    # эксперимент
    if entity_type == 'расчетный-счет-плательщика':
        if len(entity) != 20: var = True
    if entity_type == 'расчетный-счет-покупателя':
        if len(entity) != 20: var = True
    if entity_type == 'расчетный-счет-получателя':
        if len(entity) != 20: var = True
    if entity_type == 'кор-счет-банка-получателя':
        if len(entity) != 20: var = True
    if entity_type == 'кор-счет-банка-плательщика':
        if len(entity) != 20: var = True
    if entity_type == 'лицевой-счет-исполнителя':
        if len(entity) != 20: var = True
    if entity_type == 'бик-банка-плательщика':
        if len(entity) != 9: var = True
    if entity_type == 'бик-банка-получателя':
        if len(entity) != 9: var = True
    if entity_type == 'содержание-хоз-операции':
        if len(entity) < 7 and entity != 'ГСМ': var = True
    if entity_type == 'сумма-прописью':
        if len(entity) < 13: var = True
    if entity_type == 'ADDROBJ':
        if len(entity) < 12: var = True
    if entity_type == 'наименование-плательщика':
        if len(entity) < 10: var = True
    if entity_type == 'TOTALWITHNDS':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
    if entity_type == 'дата-акта':
        if len(entity) < 10: var = True
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
    if entity_type == 'наименование-получателя':
        if len(entity) < 10: var = True
    if entity_type == 'SHIPPERBANK':
        if len(entity) < 12: var = True
    if entity_type == 'банк-плательщика':
        if entity == ' БАНК ПАО': var = True
    if entity_type == 'банк-получателя':
        if len(entity) < 7: var = True
    if entity_type == 'номер-акта':
        if re.sub(pattern, '', ''.join(entity.split())).isalpha(): var = True
    return var

In [ ]:
ner_labels_dict = {}
for i in docs_df['img_names']:
    text = docs_df['text'].loc[docs_df['img_names'] == i].values[0]
    if i in labels_for_any_docs_dict:
        entities_list = []
        for j in labels_for_any_docs_dict[i]:
            entity_type, entity = j
            word_list = []
            entity_text = ''.join(entity)
            for word in entity:
                word = complex_estimation(word, text)
                if word != None and word not in punctuation:
                    word_list.append(word)
            if len(word_list) != 0 and ''.join(word_list) in text:
                entity_text = ''.join(word_list)
                if bad_word(entity_type, entity_text) == False:
                    entity_text_length = len(entity_text)
                    start_index = text.find(entity_text)
                    entities_list.append({'entity_type': entity_type, 'entity_text': entity_text, 'start': start_index, 'end': start_index + entity_text_length})
            elif entity_text in text:
                entity_text = complex_estimation(entity_text, text)
                if entity_text != None and entity_text not in punctuation and bad_word(entity_type, entity_text) == False:
                    entity_text_length = len(entity_text)
                    start_index = text.find(entity_text)
                    entities_list.append({'entity_type': entity_type, 'entity_text': entity_text, 'start': start_index, 'end': start_index + entity_text_length})
        ner_labels_dict[i] = {'text': text, 'entities': entities_list}

In [ ]:
ner_labels_dict

In [ ]:
lens_list = []
for i in ner_labels_dict:
    for j in ner_labels_dict[i]['entities']:
        if j['entity_type'] == 'номер-акта':
            lens_list.append(len(j['entity_text']))
            if len(j['entity_text']) < 100:
                print(j['entity_text'])

##### визуализация распределений длин некоторых сущностей

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# plt.xticks(range(0, 50, 2))
sns.histplot(lens_list)

### Процесс токенизации

In [ ]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item['text']))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['text'])
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item['entities']:
        e_words = sorted({idx for idx in char2word[e['start']:e['end']] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e['entity_type']
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e['entity_type']

    return {'tokens': words, 'tags': word_labels}

##### проверка

In [ ]:
import random

random.choice(list(ner_labels_dict.keys()))

In [ ]:
e = ner_labels_dict['10393498_1_page000.jpg']
for i in range(len(extract_labels(e)['tokens'])):
    if extract_labels(e)['tags'][i] != 'O':
        print(extract_labels(e)['tokens'][i], extract_labels(e)['tags'][i])

разбиение данных на тренировочную и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for _, item in ner_labels_dict.items()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [ ]:
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

In [ ]:
label_list = sorted({label for item in ner_data for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

#### Составим датасет

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

In [ ]:
from transformers import AutoTokenizer

checkpoint = "rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(ner_data['train'][22:23])

In [ ]:
# токенизация датасета

tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)
tokenized_datasets

### Fine-tuning

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    labels = [j for i in true_labels for j in i]
    preds = [j for i in true_predictions for j in i]
    return {'Accuracy': accuracy_score(labels, preds),
            'F1': f1_score(labels, preds, average="weighted")}

In [ ]:
batch_size = 5

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.train()

##### проверка

In [ ]:
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average')

In [ ]:
# вспомогательная функция для удобного отображения предсказания
def get_pred(idx=None):
    if idx == None:
        idx = random.choice(range(len(ner_test)))
    text = ' '.join(ner_test[idx]['tokens'])
    print(text)
    for i in pipe(text):
        print(i)

In [ ]:
get_pred()

In [ ]:
for i in range(2):
    get_pred()
    print('\n\n')